In [2]:
import numpy as pd
import pandas as pd
from collections import Counter
from copy import copy
import tensorflow as tf
from keras import Sequential
from keras.layers import Input, Dense, Dropout
from keras.models import Model

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Read game_based dataset
dataset_fp = "drive/Team Drives/Deep Learning Project/ann/aggregated_game_based_36_features_70min.csv"
df1 = pd.read_csv(dataset_fp)
df1.head(5)

,Corner_Kick0,Corner_Kick1,Failed_Through_Ball0,Failed_Through_Ball1,First_Yellow_Card0,First_Yellow_Card1,Foul0,Foul1,Free_Kick_Awarded0,Free_Kick_Awarded1,...,Straight_Red_Card1,Substitution0,Substitution1,assist_method0,assist_method1,fast_break0,fast_break1,id_odsp,is_goal0,is_goal1
0,7,4,0,0,0,3,9,13,12,9,...,0,0,2,14,3,1,0,UFot0hit/,3,0
1,3,4,0,0,2,2,20,10,11,19,...,0,0,2,7,9,1,0,Aw5DflLH/,2,1
2,4,3,0,0,0,1,7,20,21,9,...,0,2,2,15,6,0,0,bkjpaC6n/,1,0
3,5,3,0,0,1,1,5,17,18,7,...,0,1,1,11,15,0,0,CzPV312a/,1,0
4,2,8,0,0,0,1,6,7,7,6,...,0,1,1,9,11,0,0,GUOdmtII/,1,0


In [4]:
# add the league feature
dataset_fp = "drive/Team Drives/Deep Learning Project/ginf.csv"
df3 = pd.read_csv(dataset_fp)
df3.head()

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN


In [5]:
from tqdm import tqdm
odd_h = []
odd_d = []
odd_a = []
for i in tqdm(df1['id_odsp']):
  odd_h.append(float(df3['odd_h'][df3['id_odsp']==i]))
  odd_d.append(float(df3['odd_d'][df3['id_odsp']==i]))
  odd_a.append(float(df3['odd_a'][df3['id_odsp']==i]))
  
df3 = pd.DataFrame({'odd_h':odd_h,'odd_d':odd_d,'odd_a':odd_a})

100%|██████████| 9074/9074 [00:38<00:00, 237.26it/s]


In [0]:
df3.head()

df3['id_odsp'] = df1['id_odsp']

In [7]:
# Read game_based dataset
dataset_fp = "drive/Team Drives/Deep Learning Project/ann/team based data.csv"
df2 = pd.read_csv(dataset_fp)
df2.head(5)

,#games,goals,losing scores,loss,team,tie,wins,winning rate,loss rate,average goals,average losing score,tie rate
0,179,192,300,89,Hamburg SV,42,48,0.268156,0.497207,1.072626,1.675978,0.234637
1,180,392,197,37,Borussia Dortmund,37,106,0.588889,0.205556,2.177778,1.094444,0.205556
2,178,204,246,73,FC Augsburg,49,56,0.314607,0.410112,1.146067,1.382022,0.275281
3,132,162,205,55,SC Freiburg,35,42,0.318182,0.416667,1.227273,1.553030,0.265152
4,179,254,348,81,Werder Bremen,49,49,0.273743,0.452514,1.418994,1.944134,0.273743


Then we aggregate the two datasets

In [8]:
# create the game-based result dataset
import pandas as pd

df = pd.read_csv("drive/Team Drives/Deep Learning Project/events.csv")
df = df.ix[:,[0,8,9,16]]
game = df['id_odsp'].unique()
game_base = df.groupby('id_odsp')

game_dic = {}
for i in game:
    game_dic[i]=0
    
for each in game:
    this_game = game_base.get_group(each)
    this_game_goal = this_game.groupby('is_goal')


    if len(this_game['is_goal'].unique()) ==2:
        goal = this_game_goal.get_group(1)
        if len(goal['event_team'].unique())==2:
            a = dict(Counter(goal['event_team']))
            temp = copy(a)
            if list(temp.values())[0] > list(temp.values())[1]:
                a['win']=list(temp.keys())[0]
                a['loss']=list(temp.keys())[1]
                a['draw']=None
            else:
                if list(temp.values())[0] == list(temp.values())[1]:
                        a['win']=None
                        a['loss']=None
                        a['draw']=list(temp.keys())
                else:
                    a['win']=list(temp.keys())[1]
                    a['loss']=list(temp.keys())[0]
                    a['draw']=None
            game_dic[each]=a
        else:
            b = dict(Counter(goal['event_team']))
            temp = copy(b)
            other_team = list(this_game['event_team'].unique())
            for i in other_team:
                if list(temp.keys())[0] != i:
                    b[i]=0
                    opponent = i
            b['win'] = list(temp.keys())[0]
            b['loss'] = opponent
            b['draw'] = None
            game_dic[each]=b
    else:
        team = this_game['event_team'].unique()
        c = {}
        for i in team:

            c[i]=0
        temp = copy(c)
        c['win']=None
        c['loss']=None
        c['draw']=list(temp.keys())
        game_dic[each]=c



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [9]:
# get the result dataset and then aggregate all the data set. We have 25 features in total. For each game, add the wining rate, loss rate, tie rate, avg lossing score, avg goal for the team and opponent
id_odsp = list(game_dic.keys())
result = []
team = []
opponent = []
winning_rate0 = []
loss_rate0= []
average_goal0 = []
average_lossing_score0 =[]
tie_rate0 = []
winning_rate1 = []
loss_rate1= []
average_goal1 = []
tie_rate1 = []
average_lossing_score1 = []
for id in id_odsp:
    game = game_dic[id]
    team_temp = list(game.keys())[0]
    opponent_temp = list(game.keys())[1]
    team.append(team_temp)
    opponent.append(opponent_temp)
    if team_temp == game['win']:
        result.append(0)
    else:
        if opponent_temp == game['win']:
            result.append(1)
        else:
            result.append(2)
for i in team:
    temp = df2[df2['team']==i]
    winning_rate0.append(float(temp['winning rate']))
    loss_rate0.append(float(temp['loss rate']))
    average_goal0.append(float(temp['average goals']))
    average_lossing_score0.append(float(temp['average losing score']))
    tie_rate0.append(float(temp['tie rate']))
for j in opponent:
    temp = df2[df2['team']==j]
    winning_rate1.append(float(temp['winning rate']))
    loss_rate1.append(float(temp['loss rate']))
    average_goal1.append(float(temp['average goals']))
    average_lossing_score1.append(float(temp['average losing score']))
    tie_rate1.append(float(temp['tie rate']))
    
target = pd.DataFrame({'id_odsp':id_odsp, 'result':result, 'team':team, 'opponent':opponent, 'wining_rate0':winning_rate0, 'loss_rate0':loss_rate0
                      ,'average_goal0':average_goal0,'average_lossing_score0':average_lossing_score0, 'tie_rate0':tie_rate0,'wining_rate1':winning_rate1, 'loss_rate1':loss_rate1
                      ,'average_goal1':average_goal1,'average_lossing_score1':average_lossing_score1, 'tie_rate1':tie_rate1})
target.head(5)


,average_goal0,average_goal1,average_lossing_score0,average_lossing_score1,id_odsp,loss_rate0,loss_rate1,opponent,result,team,tie_rate0,tie_rate1,wining_rate0,wining_rate1
0,2.177778,1.072626,1.094444,1.675978,UFot0hit/,0.205556,0.497207,Hamburg SV,0,Borussia Dortmund,0.205556,0.234637,0.588889,0.268156
1,1.227273,1.146067,1.553030,1.382022,Aw5DflLH/,0.416667,0.410112,FC Augsburg,2,SC Freiburg,0.265152,0.275281,0.318182,0.314607
2,1.418994,0.666667,1.944134,1.575758,bkjpaC6n/,0.452514,0.575758,Kaiserslautern,0,Werder Bremen,0.273743,0.303030,0.273743,0.121212
3,1.195238,2.149038,1.480952,0.750000,CzPV312a/,0.433333,0.105769,Paris Saint-Germain,0,Lorient,0.266667,0.211538,0.300000,0.682692
4,1.160305,1.107143,1.519084,1.500000,GUOdmtII/,0.458015,0.500000,Valenciennes,0,Caen,0.229008,0.241071,0.312977,0.258929


In [0]:
# add the CNN features
import numpy as np
loaded = np.loadtxt('drive/Team Drives/Deep Learning Project/ken_cnn/cnn_output.txt')

In [0]:
# transfer to dataframe, add a column called id_odsp for the id of the each game
cnn_features = pd.DataFrame(loaded)
cnn_features['id_odsp'] = target['id_odsp']

In [12]:
cnn_features.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,id_odsp
0,0.0,0.0,0.933279,0.0,0.0,0.0,0.138197,0.0,0.0,0.0,...,0.0,0.618562,0.0,0.0,0.0,1.267046,0.140372,1.232537,1.187938,UFot0hit/
1,0.0,0.0,0.933279,0.0,0.0,0.0,0.138197,0.0,0.0,0.0,...,0.0,0.814164,0.0,0.0,0.0,1.085315,0.299505,1.122502,0.934438,Aw5DflLH/
2,0.0,0.0,0.933279,0.0,0.0,0.0,0.138197,0.0,0.0,0.0,...,0.0,0.336448,0.0,0.0,0.0,1.128578,0.242672,0.954873,1.008459,bkjpaC6n/
3,0.0,0.0,0.933279,0.0,0.0,0.0,0.138197,0.0,0.0,0.0,...,0.0,0.690345,0.0,0.0,0.0,1.215017,0.439924,1.354998,1.111901,CzPV312a/
4,0.0,0.0,0.933279,0.0,0.0,0.0,0.138197,0.0,0.0,0.0,...,0.0,0.806475,0.0,0.0,0.0,1.185904,0.213813,1.290539,1.014726,GUOdmtII/


In [13]:
# then add the RNN features

loaded = np.loadtxt('drive/Team Drives/Deep Learning Project/rnn_pred.txt')

loaded.shape

(9073, 36)

In [0]:
# aggregate to generate the second half features for each game
rnn_features = pd.DataFrame(loaded, columns = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','a1','b1','c1','d1','e1','f1','g1','h1','i1','j1','k1','l1','m1','n1','o1','p1','q1','r1'])
rnn_features['id_odsp'] = target['id_odsp']



In [15]:
rnn_features.head()

,a,b,c,d,e,f,g,h,i,j,...,j1,k1,l1,m1,n1,o1,p1,q1,r1,id_odsp
0,0.050860,0.056245,0.000062,0.000052,0.026439,0.029196,0.135642,0.139206,0.143163,0.139593,...,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902,UFot0hit/
1,0.050861,0.056245,0.000062,0.000052,0.026439,0.029196,0.135642,0.139207,0.143163,0.139594,...,0.000815,0.056525,0.056886,0.151884,0.156790,0.004086,0.001938,0.017707,0.010902,Aw5DflLH/
2,0.050861,0.056245,0.000062,0.000052,0.026439,0.029196,0.135642,0.139207,0.143163,0.139594,...,0.000815,0.056525,0.056886,0.151884,0.156790,0.004086,0.001938,0.017707,0.010902,bkjpaC6n/
3,0.050860,0.056245,0.000062,0.000052,0.026439,0.029196,0.135642,0.139206,0.143163,0.139593,...,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902,CzPV312a/
4,0.050861,0.056245,0.000062,0.000052,0.026439,0.029196,0.135642,0.139208,0.143163,0.139594,...,0.000815,0.056525,0.056886,0.151884,0.156790,0.004086,0.001938,0.017707,0.010902,GUOdmtII/


In [16]:
cnn_features.index[0]

0

In [17]:
# merge the data to get the final dataset
final1 = pd.merge(df1,target)
final2 = pd.merge(final1, cnn_features)
final2 = pd.merge(final2, df3)
final3 = pd.merge(final2, rnn_features)
# final.to_csv("drive/Team Drives/Deep Learning Project/feature_together.csv", index = False)

final3.head(10)

,Corner_Kick0,Corner_Kick1,Failed_Through_Ball0,Failed_Through_Ball1,First_Yellow_Card0,First_Yellow_Card1,Foul0,Foul1,Free_Kick_Awarded0,Free_Kick_Awarded1,...,i1,j1,k1,l1,m1,n1,o1,p1,q1,r1
0,7,4,0,0,0,3,9,13,12,9,...,0.000603,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902
1,3,4,0,0,2,2,20,10,11,19,...,0.000603,0.000815,0.056525,0.056886,0.151884,0.156790,0.004086,0.001938,0.017707,0.010902
2,4,3,0,0,0,1,7,20,21,9,...,0.000603,0.000815,0.056525,0.056886,0.151884,0.156790,0.004086,0.001938,0.017707,0.010902
3,5,3,0,0,1,1,5,17,18,7,...,0.000603,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902
4,2,8,0,0,0,1,6,7,7,6,...,0.000603,0.000815,0.056525,0.056886,0.151884,0.156790,0.004086,0.001938,0.017707,0.010902
5,3,1,0,0,2,2,21,18,20,21,...,0.000603,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902
6,7,5,0,0,2,1,11,8,8,14,...,0.000603,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902
7,4,10,0,0,1,1,13,10,10,13,...,0.000603,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902
8,11,5,0,0,0,1,10,9,9,10,...,0.000603,0.000815,0.056525,0.056886,0.151883,0.156789,0.004086,0.001938,0.017707,0.010902
9,3,7,0,0,2,1,13,8,9,15,...,0.000603,0.000815,0.056525,0.056886,0.151884,0.156791,0.004086,0.001938,0.017707,0.010902


In [0]:
# set up the training and test set
del final3['id_odsp']


dic_team = {}
count1 = 0

for i in final3['team'].unique():
  dic_team[i] = count1
  count1 +=1
final3['team'] = [dic_team[i] for i in final3['team']]
final3['opponent'] = [dic_team[i] for i in final3['opponent']]

from sklearn.model_selection import train_test_split
X= final3.drop('result', axis =1)
Y = final3['result']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

models

In [19]:
len(X.columns)

343

In [20]:
len(final3)

9073

In [21]:
# Set parameters
input_number = 343
# --- Setting up a Sigmoid Sequential Model ---
# Initialize model
model = Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(Dense(128, input_shape = (input_number,), activation='relu'))
model.add(Dropout(0.2))
# Add another:
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
##model.add(Dropout(0.2))
##model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
# Add a softmax layer with 10 output units:
model.add(Dense(3, activation='softmax'))
# Check model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               44032     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 60,931
Trainable params: 60,931
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
Y = to_categorical(Y)

In [23]:

import keras
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X,Y, epochs =20,batch_size = 32, validation_split = 0.2)             


Train on 7258 samples, validate on 1815 samples
Epoch 1/20
7258/7258 [==============================] - 4s 509us/step - loss: 1.1079 - acc: 0.5875 - val_loss: 0.8303 - val_acc: 0.6474
Epoch 2/20
7258/7258 [==============================] - 2s 217us/step - loss: 0.8038 - acc: 0.6414 - val_loss: 0.7662 - val_acc: 0.6562
Epoch 3/20
7258/7258 [==============================] - 2s 220us/step - loss: 0.7237 - acc: 0.6773 - val_loss: 0.6824 - val_acc: 0.6815
Epoch 4/20
7258/7258 [==============================] - 2s 214us/step - loss: 0.6730 - acc: 0.6955 - val_loss: 0.6449 - val_acc: 0.7135
Epoch 5/20
7258/7258 [==============================] - 2s 218us/step - loss: 0.6443 - acc: 0.7133 - val_loss: 0.6348 - val_acc: 0.7124
Epoch 6/20
7258/7258 [==============================] - 2s 214us/step - loss: 0.6274 - acc: 0.7202 - val_loss: 0.6826 - val_acc: 0.6909
Epoch 7/20
7258/7258 [==============================] - 2s 216us/step - loss: 0.6121 - acc: 0.7316 - val_loss: 0.6267 - val_acc: 0.7080


In [24]:
scores = model.evaluate(X_test, y_test) 
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

2995/2995 [==============================] - 0s 50us/step
acc: 76.06%


In [25]:
top_ten = ['Barcelona',
 'Juventus',
 'Real Madrid',
 'Paris Saint-Germain',
 'Bayern Munich',
 'Atletico Madrid',
 'Napoli',
 'AS Roma',
 'Lyon',
 'Borussia Dortmund']

# PREDICTIONS FOR THE TOP TEAMS
# merge the data to get the final dataset
final1 = pd.merge(df1,target)
final2 = pd.merge(final1, cnn_features)
final2 = pd.merge(final2, df3)
final3 = pd.merge(final2, rnn_features)
# final.to_csv("drive/Team Drives/Deep Learning Project/feature_together.csv", index = False)

del final3['id_odsp']
final3_10 = final3
final3_5 = final3
final3_3 = final3
final3_1 = final3

for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten and final3['opponent'][i] not in top_ten:
        final3_10 = final3_10.drop(i)

for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten[0:5] and final3['opponent'][i] not in top_ten[0:5]:
        final3_5 = final3_5.drop(i)
for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten[0:3] and final3['opponent'][i] not in top_ten[0:3]:
        final3_3 = final3_3.drop(i)
for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten[0] and final3['opponent'][i] not in top_ten[0]:
        final3_1 = final3_1.drop(i)
        

100%|██████████| 9073/9073 [00:46<00:00, 193.52it/s]


In [26]:

from sklearn.model_selection import train_test_split
final3_10_ = final3_10.reset_index(drop = True)
final3_5_ = final3_5.reset_index(drop = True)
final3_3_ = final3_3.reset_index(drop = True)
final3_1_ = final3_1.reset_index(drop = True)

m = [final3_10_, final3_5_, final3_3_, final3_1_]
name = ['10','5','3','1']
for j in range(4):
  m[j]['team'] = [dic_team[i] for i in m[j]['team']]
  m[j]['opponent'] = [dic_team[i] for i in m[j]['opponent']]


  X= m[j].drop('result', axis =1)
  Y = m[j]['result']

  Y = to_categorical(Y)
  score = model.evaluate(X,Y)
  print('For top', name[j],'teams' )
  print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

1910/1910 [==============================] - 0s 51us/step
For top 10 teams
acc: 82.15%
985/985 [==============================] - 0s 53us/step
For top 5 teams
acc: 84.47%
598/598 [==============================] - 0s 61us/step
For top 3 teams
acc: 83.78%
201/201 [==============================] - 0s 78us/step
For top 1 teams
acc: 83.58%


In [0]:
# save 
model.save('drive/Team Drives/Deep Learning Project/ann/final_model.h5')